In [137]:
import pandas as pd

trade = pd.read_csv(r"C:\Users\queiy\trading_algorithm\pnl_process\trade_sample.csv")
trade = trade.set_index("timestamp")
trade = trade.fillna(0)
trade.index = pd.to_datetime(trade.index, format = "%d/%m/%Y")
trade.head()


,y,y_pos
timestamp,,
2014-07-08,108.94,0.0
2014-07-09,108.28,0.0
2014-07-10,108.67,100.0
2014-07-11,106.66,100.0
2014-07-14,106.98,100.0


In [79]:
import numpy as np
df = trade.copy()
df["long_short"] = (df.y_pos > 0) * 1  -  (df.y_pos < 0) * 1

trade_count = 0
trade_detail = []
long_short = 0
    
for i, data_slice in enumerate(df.iterrows()):
    s = data_slice[1]

    if i > 1 and s.long_short != df.iloc[i-1].long_short:
        if long_short != 0:
            print(i, "Close")
            close_price = s.y
            close_date = s.name
            trade_detail.append({"trade": trade_count,
                            "position": position,
                            "open_date": open_date,
                            "open_price": open_price,
                            "close_date": close_date,
                            "close_price": s.y})
            long_short = 0
            
        if s.long_short != 0:
            print(i, "Open")
            open_price = s.y
            position = s.y_pos
            open_date = s.name
            trade_count += 1
            long_short = s.long_short
        
    if s.long_short != long_short:
        print(i, "Close")
        close_price = s.y
        close_date = s.name
        trade_detail.append({"trade": trade_count,
                            "position": position,
                            "open_date": open_date,
                            "open_price": open_price,
                            "close_date": close_date,
                            "close_price": s.y})
        

pd.DataFrame(trade_detail)

2 Open
6 Close
6 Open
11 Close
12 Open
19 Close


,trade,position,open_date,open_price,close_date,close_price
0,1,100.0,10/07/2014,108.67,16/07/2014,105.85
1,2,-200.0,16/07/2014,105.85,23/07/2014,108.03
2,3,50.0,24/07/2014,107.07,04/08/2014,105.41


In [138]:
import numpy as np
df = trade.copy()
df["long_short"] = (df.y_pos > 0) * 1  -  (df.y_pos < 0) * 1

def trade_summary(df):
    trade_detail = []
    def update_trade(s, trade_count, position, open_date, open_price, close_date, close_price):
        trade_detail.append({"trade": trade_count,
                                "position": position,
                                "open_date": open_date,
                                "open_price": open_price,
                                "close_date": close_date,
                                "close_price": s.y,
                                "realized_pnl": position * (s.y - open_price)})
    trade_count = 0
    long_short = 0
    
    for i, data_slice in enumerate(df.iterrows()):
        s = data_slice[1]

        if i > 1 and s.long_short != df.iloc[i-1].long_short:
            if long_short != 0:
                print(i, "Close")
                close_price, close_date = s.y, s.name
                update_trade(s, trade_count, position, open_date, open_price, close_date, s.y)
                long_short = 0

            if s.long_short != 0:
                print(i, "Open")
                open_price = s.y
                position = s.y_pos
                open_date = s.name
                trade_count += 1
                long_short = s.long_short

        if s.long_short != long_short:
            print(i, "Close")
            close_price, close_date = s.y, s.name
            close_date = s.name
            update_trade(s, trade_count, position, open_date, open_price, close_date, s.y)

    return pd.DataFrame(trade_detail)

trade_summary(df)

2 Open
6 Close
6 Open
11 Close
12 Open
19 Close


,trade,position,open_date,open_price,close_date,close_price,realized_pnl
0,1,100.0,2014-07-10,108.67,2014-07-16,105.85,-282.0
1,2,-200.0,2014-07-16,105.85,2014-07-23,108.03,-436.0
2,3,50.0,2014-07-24,107.07,2014-08-04,105.41,-83.0


In [135]:
import pandas as pd


def trade_summary(df_input, security_name, position_name):
    """
    For static positions only, i.e. at any time a fixed unit of positions are live.
    Take a dataframe with timestamp index, security, and security_pos (position) and calculate PnL trade by trade.
    """
    df = df_input.copy()
    df["long_short"] = (df[position_name] > 0) * 1 - (df[position_name] < 0) * 1

    trade_detail = []

    def update_trade(_trade_count, _position, _open_date, _open_price, _close_date, _close_price):
        trade_detail.append({"trade": _trade_count,
                             "position": _position,
                             "open_date": _open_date,
                             "open_price": _open_price,
                             "close_date": _close_date,
                             "close_price": _close_price,
                             "realized_pnl": _position * (_close_price - open_price)})
    trade_count = 0
    long_short = 0
    position = 0
    open_date = 0
    open_price = 0
    
    for i, data_slice in enumerate(df.iterrows()):
        s = data_slice[1]   # Slice
        if i > 1 and s.long_short != df.iloc[i - 1].long_short:
            if long_short != 0:
                close_price, close_date = s[security_name], s.name
                update_trade(trade_count, position, open_date, open_price, close_date, close_price)
                long_short = 0

            if s.long_short != 0:
                open_price = s[security_name]
                position = s[position_name]
                open_date = i  # date/time from index
                trade_count += 1
                long_short = s.long_short

        if s.long_short != long_short:
            close_price, close_date = s.y, s.name
            close_date = s.name
            update_trade(trade_count, position, open_date, open_price, close_date, close_price)

    return pd.DataFrame(trade_detail)

trade = pd.read_csv(r"C:\Users\queiy\trading_algorithm\pnl_process\trade_sample.csv")
trade = trade.set_index("timestamp")
trade = trade.fillna(0)

x = trade_summary(trade, "y", "y_pos")
x

,trade,position,open_date,open_price,close_date,close_price,realized_pnl
0,1,100.0,2,108.67,16/07/2014,105.85,-282.0
1,2,-200.0,6,105.85,23/07/2014,108.03,-436.0
2,3,50.0,12,107.07,04/08/2014,105.41,-83.0


In [142]:
y = x[["close_date", "realized_pnl"]]
y = y.set_index("close_date")
y.index.name = trade.index.name
y.index = pd.to_datetime(y.index, format = "%d/%m/%Y")
y = pd.concat([y, trade], axis = 1)
y.realized_pnl = y.realized_pnl.fillna(0)
y

,realized_pnl,y,y_pos
timestamp,,,
2014-07-08,0.0,108.94,0.0
2014-07-09,0.0,108.28,0.0
2014-07-10,0.0,108.67,100.0
2014-07-11,0.0,106.66,100.0
2014-07-14,0.0,106.98,100.0
2014-07-15,0.0,106.02,100.0
2014-07-16,-282.0,105.85,-200.0
2014-07-17,0.0,107.89,-200.0
2014-07-18,0.0,107.24,-200.0
